In [1]:
# !pip install chromadb
# !pip install -Uq chromadb numpy datasets
!pip install langchain
!pip install openai
!pip install pypdf
!pip install python-dotenv
!pip install tiktoken
!pip install numpy==1.22
!pip install faiss-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 272.6/272.6 kB 3.3 MB/s eta 0:00:00


In [3]:
import os
import openai
import sys
sys.path.append('../..')

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']

In [25]:
from langchain.document_loaders import PyPDFLoader
loader = [PyPDFLoader("resumenew.pdf"),PyPDFLoader("shivam.pdf")]
docs = []
for load in loader:
  docs.extend(load.load())

In [26]:
docs[0].page_content

'Rohit Kumar \nRanchi , Jharkhand  +91-9798494187 rohitcode005@gmail.com linkedin EDUCATION EXPERIENCE Sarala Birla University \nB.Tech in Artificial Intelligence Data Engineer Intern Taiyo.ai March 2021-Present July 2023 Sep 2023 Dec 2022 Jun 2021 - Sep 2021 May 2021 - Present All India Samsung solve for tomorrow - Innovation Conduct web scraping to extract data from both the World Bank and IADB etc websites\nestablishment of a robust data pipeline for seamless integration with Google Cloud.\nEmploy Python scripting, emphasizing Object-Oriented Programming (OOP) principles\nEffectively preprocess this data using Pandas and NumPy. Predict user emotions from audio input using a Flask website trained on 2,800 audio data samples across 7\nemotions: happy, sad, surprise, fear, angry, disgust, and neutral. Flask web app that classify and Localize input image from CIFAR-10,specifically distingusing between\neggplant and cucumber Functionality: A platform for HR to upload and manage hundreds 

In [27]:
# Split
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 300,
    chunk_overlap = 50
)

In [28]:
splits = text_splitter.split_documents(docs)

In [29]:
from langchain.embeddings.openai import OpenAIEmbeddings
embedding = OpenAIEmbeddings()

In [30]:
!rm -rf ./docs/chroma  # remove old database files if any

In [31]:
persist_directory = 'docs/chroma/'

In [32]:
from langchain.vectorstores.faiss import FAISS

vectordb = FAISS.from_documents(splits, embedding)

In [33]:
# from langchain.vectorstores import Chroma
# vectordb = Chroma.from_documents(
#     documents=splits,
#     embedding=embedding,
#     persist_directory=persist_directory
# )


In [34]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI


llm_name = "gpt-3.5-turbo"
llm = ChatOpenAI(model_name=llm_name, temperature=1)

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever()
)

In [40]:
result = qa_chain({"query": "who has done MTech"})
result["result"]

'Shivam Chhirolya has done MTech in Artificial Intelligence.'